Script that 
* takes as input the files config-layers-X (X: polygon, point, linestring), and municipality codes
* for all municipality codes jointly, makes the "technical network"
* for each X,
    * determine evaluation layers
    * fetch and merge gdfs for given muni (if they exist)
    * save to `input-for-bike-node-planner/X`

In [1]:
# import libraries
import os
import yaml
import geopandas as gpd
import pandas as pd

exec(open("../src/helper_functions.py").read())

In [2]:
# read config files
config = yaml.load(
    open("../config.yml"), 
    Loader=yaml.FullLoader)
proj_crs = config["proj_crs"]

municipalities = yaml.load(
    open("../config-municipalities.yml"), 
    Loader=yaml.FullLoader)
codes = municipalities["kommunekode"]

geomtypes = ["point", "linestring", "polygon"]
config_layers = {}
for geomtype in geomtypes:
    config_layers[geomtype] = yaml.load(
        open(f"../config-layers-{geomtype}.yml"), 
        Loader=yaml.FullLoader)

In [3]:
# make folders
exec(open("./make_folders.py").read())

Folders created!


In [4]:
# remove previous output
remove_output_data(
    [
        # "../input-for-bike-node-planner/dem",
        # "../input-for-bike-node-planner/elevation",
        "../input-for-bike-node-planner/linestring/",
        "../input-for-bike-node-planner/network/",
        "../input-for-bike-node-planner/point/",
        "../input-for-bike-node-planner/polygon/",
        "../input-for-bike-node-planner/studyarea/"        
    ],
    remove_previous_output=True,
)

Data folder cleaned!


**Study area**

In [5]:
### read in municipality boundaries & create study area polygon
gdf = gpd.read_file("../data/municipality-boundaries/municipality-boundaries.gpkg")
gdf = gdf.to_crs(proj_crs) # make sure we have the right projected CRS
gdf = gdf[gdf["kommunekode"].isin(codes)] # filter to municipality codes indicated in config file
gdf_studyarea = gpd.GeoDataFrame(
    {
        "geometry": [gdf.unary_union]
    },
    crs = proj_crs
)
gdf_studyarea.to_file(
    filename = "../input-for-bike-node-planner/studyarea/studyarea.gpkg", 
    index = False)
print("Study area polygon created")
del gdf

Study area polygon created


**Network**

In [6]:
### Cut technical network to study area extent

# read in nodes & edges
nodes = gpd.read_file("../data/network-technical/nodes.gpkg")
nodes = nodes.to_crs(proj_crs)

edges = gpd.read_file("../data/network-technical/edges.gpkg")
edges = edges.to_crs(proj_crs)

# limit to extent of study area
edges_studyarea = edges.sjoin(gdf_studyarea, predicate="intersects").copy()
edges_studyarea.drop(columns = ["index_right"], inplace = True)
nodes_studyarea = nodes.clip(edges_studyarea.buffer(500).unary_union)

# save
os.makedirs("../input-for-bike-node-planner/network/technical/", exist_ok=True)
edges_studyarea.to_file("../input-for-bike-node-planner/network/technical/edges.gpkg", index = False)
nodes_studyarea.to_file("../input-for-bike-node-planner/network/technical/nodes.gpkg", index = False)

print("Nodes and edges for study area saved!")

Nodes and edges for study area saved!


## PICK UP HERE! 

**above** add preprocessing steps for technical network from `02_make_technical_layer`

**below** ADD CODE FOR TECH>COMM from `03_make_communication_layer`

**Evaluation layers**

In [7]:
# create a dictionary of evaluation layers (based on config file inputs)

layer_dict = {}

for geomtype in geomtypes:

    layer_dict[geomtype] = {}

    # determine evaluation layers
    layers = []
    for v in config_layers[geomtype].values():
        layers += list(set(v.values()))
    layers = list(set(layers))
    
    # determine data sets that go into each layer

    # key is name of merged output layer, value is a dict 
    for layer in layers:
        layer_dict[geomtype][layer] = {}
    
    # adding data source as key to dictindict IF relevant to layer
    for datasource, vdict in config_layers[geomtype].items():
        for layer in (set(vdict.values())):
            layer_dict[geomtype][layer][datasource] = []
    
    for datasource, vdict in config_layers[geomtype].items():
        for k, v in vdict.items():
            layer_dict[geomtype][v][datasource] += [k]

for geomtype in geomtypes:
    if "ignore" in layer_dict[geomtype]:
        del layer_dict[geomtype]["ignore"]

print(layer_dict)

{'point': {'facility': {'facilit_rasteplads': ['Bænk', 'Picnicbord', 'Legeplads', 'Borde-bænke/Picnic', 'Tørvejrsrum/Madpakkehus', 'Naturlegeplads', 'Vandlegeplads'], 'facilit_service': ['Toilet', 'Drikkevand', 'Cykelhandler', 'Vandpost', 'Cykeludlejere', 'Cykelservice', 'Cykelpumpestation/-servicepost']}, 'service': {'facilit_indkoeb': ['supermarked', 'bager', 'kiosk', 'slagter', 'Lokale smagsoplevelser', 'tankstation', 'fisk'], 'facilit_overnatning': ['Bed & Breakfast', 'Hotel', 'Campingplads', 'Slot/Herregård', 'Vandrehjem', 'Feriecenter', 'Kro'], 'facilit_service': ['Turistkontor']}, 'poi': {'land_besoegs': ['Museum', 'Forlystelsespark', 'Dyrepark, zoo', 'Dyrepark', 'Anden seværdighed', 'Frilandsmuseum', 'Aboret, blomsterpark o.l.'], 'land_fortid': ['Gravhøj', 'Vold eller voldsted', 'Langdysse', 'Jættestue', 'Fortidsminde/ruin', 'Skanse', 'Ruin', 'Batteri', 'Dysse', 'Runddysse', 'Andet historisk anlæg', 'Fæstningsanlæg', 'Oldtidsvej', 'Vikingeborg', 'Oldtidsminde'], 'land_landemaer

In [8]:
# for each layer type (point/linestring/polygon),
for geomtype in geomtypes:

    print(geomtype)
    
    # go through all evaluation layers for that geomtype... 
    for layername, datadict in layer_dict[geomtype].items():
    
        print("\t", layername)
        final_gdf = gpd.GeoDataFrame()
        
        # go through each data source for that evaluation layer...
        for k, v in datadict.items():

            gdf = gpd.GeoDataFrame()

            # and fetch it for each municipality
            for code in codes:
                print("\t \t", code, k)
                # for each code, check if file exists, if yes: read it in, if not empty: concatenate
                fp = f"../data/{geomtype}/{code}/{k}.gpkg"
                if os.path.exists(fp):
                    gdf_muni = gpd.read_file(fp)
                    if not gdf_muni.empty:
                        gdf = pd.concat(
                            [
                                gdf,
                                gdf_muni
                            ]
                        )

            # if at least one of the municipalities has data from this data source,
            # add it to final gdf
            if not gdf.empty:
                final_gdf = pd.concat(
                    [
                        final_gdf,
                        gdf[gdf["type"].isin(v)]
                    ]
                )

        # save evaluation layer to file, if not empty
        if not final_gdf.empty:
            final_gdf = final_gdf.reset_index(drop=True)
            final_gdf.to_file(
                f"../input-for-bike-node-planner/{geomtype}/{layername}.gpkg", 
                index = False
            )
            print("\t", f"{layername} saved")
        else:
            print("\t", f"No data found for layer {layername}")

point
	 facility
	 	 0320 facilit_rasteplads
	 	 0320 facilit_service
	 facility saved
	 service
	 	 0320 facilit_indkoeb
	 	 0320 facilit_overnatning
	 	 0320 facilit_service
	 service saved
	 poi
	 	 0320 land_besoegs
	 	 0320 land_fortid
	 	 0320 land_landemaerke
	 	 0320 land_udflugt
	 poi saved
linestring
polygon
	 sommerhus
	 	 0320 land_anvendelse
	 sommerhus saved
	 verify
	 	 0320 land_anvendelse
	 verify saved
	 nature
	 	 0320 land_anvendelse
	 	 0320 land_beskyttnatur_flade
	 	 0320 land_frednatpark
	 	 0320 land_vaerdifuld
	 nature saved
	 culture
	 	 0320 land_anvendelse
	 	 0320 land_vaerdifuld
	 culture saved
	 agriculture
	 	 0320 land_anvendelse
	 agriculture saved


**Elevation**

In [9]:
exec(open("./prepare_elevation_data.py").read())

Data folder cleaned!
Elevation data downloaded! Starting to merge...


/Users/anvy/anaconda3/envs/knupu/lib/python3.11/site-packages/osgeo/gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Elevation data ready!


In [13]:
print("All done! Now you can copy-paste \nall subfolders of '/input-for-bike-node-planner/' \ninto the '/data/input/' folder of the main repo")

All done! Now you can copy-paste 
all subfolders of '/input-for-bike-node-planner/' 
into the '/data/input/' folder of the main repo
